In [3]:
import os; os.chdir('..')
import sys; sys.path.extend(['.'])
os.environ['CC'] = 'gcc-10'
os.environ['CXX'] = 'g++-10'

In [4]:
from PIL import Image

import os
from PIL import Image
import argparse
from tqdm import tqdm
import torch
import torchvision.transforms.functional as TVF
from torchsummary import summary
import numpy as np
from distutils.dir_util import copy_tree
from training.networks import SynthesisLayer
from training.networks import PatchWiseSynthesisLayer
import dnnlib
from scripts.legacy import load_network_pkl

np.random.seed(42)
torch.manual_seed(42)
torch.set_grad_enabled(False)

np.float = float
# np.int = int   #module 'numpy' has no attribute 'int'
# np.object = object    #module 'numpy' has no attribute 'object'
# np.bool = bool    #module 'numpy' has no attribute 'bool'

network_pkl = 'file:///home/chris/projects/ml/alis/lhq1024-snapshot.pkl'
device = 'cuda'

with dnnlib.util.open_url(network_pkl) as f:
    G = load_network_pkl(f)['G_ema'].to(device) # type: ignore
    G.eval()
    G.progressive_growing_update(100000)

for res in G.synthesis.block_resolutions:
    block = getattr(G.synthesis, f'b{res}')
    if hasattr(block, 'conv0'):
        block.conv0.use_noise = False
    block.conv1.use_noise = False

/tmp/ipykernel_73934/2674653408.py:11: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree


In [5]:
num_frames = 16
num_frames_per_w = G.synthesis_cfg.patchwise.w_coord_dist // 2
num_ws = num_frames // num_frames_per_w + 1
shifts = torch.arange(num_frames) * G.synthesis_cfg.patchwise.grid_size
w_range = 2 * num_frames_per_w * G.synthesis_cfg.patchwise.grid_size
max_shift = (num_frames_per_w * 2 - 1) * G.synthesis_cfg.patchwise.grid_size
zs = torch.randn(num_ws, G.z_dim).to(device) 
mode_idx = 0
modes_idx = (torch.ones(1, device=zs.device).repeat(num_ws).float() * mode_idx).long()
ws = G.mapping(zs, c=None, modes_idx=modes_idx)

z_mean = torch.randn(1000, G.z_dim).to(device)
ws_proto = G.mapping(z_mean, c=None, modes_idx=modes_idx[0]).mean(dim=0, keepdim=True)

# Truncating
truncation_factor = 1.0
ws = ws * truncation_factor + (1 - truncation_factor) * ws_proto

imgs = []
curr_w_idx = 1
curr_ws = ws[curr_w_idx].unsqueeze(0)
curr_ws_context = torch.stack([ws[curr_w_idx - 1].unsqueeze(0), ws[curr_w_idx + 1].unsqueeze(0)], dim=1)

for shift in shifts:
    if shift > 0 and shift % w_range == 0:
        curr_w_idx += 2
        curr_ws = ws[curr_w_idx].unsqueeze(0)
        curr_ws_context = torch.stack([ws[curr_w_idx - 1].unsqueeze(0), ws[curr_w_idx + 1].unsqueeze(0)], dim=1)

    curr_left_borders_idx = torch.zeros(1, device=zs.device).long() + (shift % w_range)
    img = G.synthesis(curr_ws, ws_context=curr_ws_context, left_borders_idx=curr_left_borders_idx, noise='const')
    imgs.append(img[0].cpu().clamp(-1, 1) * 0.5 + 0.5)

whole_img = torch.cat(imgs, dim=2)
TVF.resize(TVF.to_pil_image(whole_img), 256, interpolation=Image.Resampling.LANCZOS)

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


KeyboardInterrupt: 

### Generating a video

In [ ]:
import os
from PIL import Image
import torch
import torchvision.transforms.functional as TVF

import numpy as np
import scipy.misc
import cv2
from tqdm import tqdm

# Load data
frame_size = 512 # Determines the height (in pixels) of the video
frames = [TVF.to_pil_image(img) for img in imgs]
frames = [TVF.resize(img, frame_size, interpolation=Image.Resampling.LANCZOS) for img in frames]

step_size = 2 # Controls the camera movement speed (i.e. how many pixels we move from frame to frame)
aspect_ratio = 4 # Aspect ratio of the video
h = frame_size
w = frame_size * aspect_ratio
whole_img = torch.cat([TVF.to_tensor(im) for im in frames], dim=2)
num_frames = (whole_img.shape[2] - frame_size * aspect_ratio) // step_size
curr_offset = 0

save_path = 'video.mp4'
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter(save_path, fourcc, 60, (w, h))
for frame_idx in tqdm(range(num_frames)):
    curr_offset += step_size
    frame = whole_img[:, :, curr_offset:curr_offset + frame_size * aspect_ratio]
    frame = TVF.to_pil_image(frame)
    video.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))

# Uncomment this line to release the memory.
# It didn't work for me on centos and complained about installing additional libraries (which requires root access)
# cv2.destroyAllWindows()
video.release()

/tmp/ipykernel_26709/3450232161.py:14: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  frames = [TVF.resize(img, frame_size, interpolation=Image.LANCZOS) for img in frames]
 27%|██▋       | 820/3072 [00:36<01:55, 19.46it/s]